
README
1.   개인 구글 드라이브에 모든 파일을 업로드
2.   해당 주피터노트 파일을 실행
3.   선언한 문제가 있는 파일에 Task plan 생성








**구글 드라이브와 연동 **

드라이브 연결이 한번에 안된다면 한번 더 인증 절차를 밟으면 연동

In [1]:
# 구글 드라이브와 연동 인증
from google.colab import auth
auth.authenticate_user()
# 구글 드라이브의 root가 '/content/grive/My Drive'로 mount 됨
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/pyperplan-master

Mounted at /content/gdrive
/content/gdrive/MyDrive/pyperplan-master



***라이브러리 설치***

In [2]:
!python setup.py install
%cd src

running install
running bdist_egg
running egg_info
writing src/pyperplan.egg-info/PKG-INFO
writing dependency_links to src/pyperplan.egg-info/dependency_links.txt
writing entry points to src/pyperplan.egg-info/entry_points.txt
writing requirements to src/pyperplan.egg-info/requires.txt
writing top-level names to src/pyperplan.egg-info/top_level.txt
reading manifest file 'src/pyperplan.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'src/pyperplan.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
copying build/lib/run.py -> build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pyperplan
copying build/lib/pyperplan/grounding.py -> build/bdist.linux-x86_64/egg/pyperplan
copying build/lib/pyperplan/planner.py -> build/bdist.linux-x86_64/egg/pyperplan
copying build/lib/pyperplan/__init__.py -> build/bdist.linux-x86_64/egg/pyperplan
copying b

In [5]:
!python run.py --help

usage: run.py [-h] [-l {debug,info,warning,error}]
              [-H {lmcut,landmark,hadd,hff,hmax,hsa,blind}]
              [-s {astar,wastar,gbf,bfs,ehs,ids,sat}]
              [domain] problem

positional arguments:
  domain
  problem

optional arguments:
  -h, --help            show this help message and exit
  -l {debug,info,warning,error}, --loglevel {debug,info,warning,error}
  -H {lmcut,landmark,hadd,hff,hmax,hsa,blind}, --heuristic {lmcut,landmark,hadd,hff,hmax,hsa,blind}
                        Select a heuristic (default: hff)
  -s {astar,wastar,gbf,bfs,ehs,ids,sat}, --search {astar,wastar,gbf,bfs,ehs,ids,sat}
                        Select a search algorithm from astar, weighted astar,
                        greedy best first, breadth first, enforced
                        hillclimbing, iterative deepening, sat solve (default:
                        bfs)


***라이브러리 추가***

In [3]:
import argparse
import logging
import os
import sys

from pyperplan import *
from pyperplan import tools
from pyperplan.planner import (
    find_domain,
    HEURISTICS,
    search_plan,
    SEARCHES,
    validate_solution,
    write_solution,
)

***Argument 선언***

In [13]:
# 도메인 정의
domain = "/content/gdrive/MyDrive/pyperplan-master/benchmarks/blocks/domain.pddl"  # 이곳을 수정하여 도메인 교체
# 문제 정의
problem = "/content/gdrive/MyDrive/pyperplan-master/benchmarks/blocks/task01.pddl"  # 이곳을 수정하여 문제 교체

# 휴리스틱 정의
heuristic = "hff"

# 탐색 알고리즘 정의
search = "astar"

#로그 수준 정의
loglevel = "info"

***실행 코드***

In [14]:

log_levels = ["debug", "info", "warning", "error"]


def get_callable_names(callables, omit_string):
    names = [c.__name__ for c in callables]
    names = [n.replace(omit_string, "").replace("_", " ") for n in names]
    return ", ".join(names)

search_names = get_callable_names(SEARCHES.values(), "_search")


logging.basicConfig(
    level=getattr(logging, loglevel.upper()),
    format="%(asctime)s %(levelname)-8s %(message)s",
    stream=sys.stdout,
)

hffpo_searches = ["gbf", "wastar", "ehs"]
if heuristic == "hffpo" and search not in hffpo_searches:
    print(
        "ERROR: hffpo can currently only be used with %s\n" % hffpo_searches,
        file=sys.stderr,
    )
    sys.exit(2)

problem = os.path.abspath(problem)
if domain is None:
    domain = find_domain(problem)
else:
    domain = os.path.abspath(domain)

searchs = SEARCHES[search]
heuristics = HEURISTICS[heuristic]

if search in ["bfs", "ids", "sat"]:
    heuristics = None

logging.info("using search: %s" % searchs.__name__)
logging.info("using heuristic: %s" % (heuristics.__name__ if heuristics else None))
use_preferred_ops = heuristic == "hffpo"
solution = search_plan(
    domain,
    problem,
    searchs,
    heuristics,
    use_preferred_ops=use_preferred_ops,
)

if solution is None:
    logging.warning("No solution could be found")
else:
    solution_file = problem + ".soln"
    logging.info("Plan length: %s" % len(solution))
    write_solution(solution, solution_file)
    validate_solution( domain, problem, solution_file)

2021-03-12 14:49:51,191 INFO     using search: astar_search
2021-03-12 14:49:51,193 INFO     using heuristic: hFFHeuristic
2021-03-12 14:49:51,194 INFO     Parsing Domain /content/gdrive/MyDrive/pyperplan-master/benchmarks/blocks/domain.pddl
2021-03-12 14:49:51,199 INFO     Parsing Problem /content/gdrive/MyDrive/pyperplan-master/benchmarks/blocks/task12.pddl
2021-03-12 14:49:51,202 INFO     5 Predicates parsed
2021-03-12 14:49:51,203 INFO     4 Actions parsed
2021-03-12 14:49:51,205 INFO     7 Objects parsed
2021-03-12 14:49:51,206 INFO     0 Constants parsed
2021-03-12 14:49:51,207 INFO     Grounding start: blocks-7-2
2021-03-12 14:49:51,214 INFO     Relevance analysis removed 0 facts
2021-03-12 14:49:51,216 INFO     Grounding end: blocks-7-2
2021-03-12 14:49:51,217 INFO     71 Variables created
2021-03-12 14:49:51,219 INFO     112 Operators created
2021-03-12 14:49:51,221 INFO     Search start: blocks-7-2
2021-03-12 14:49:51,224 INFO     Initial h value: 12.000000
2021-03-12 14:49:5

# 새 섹션

**드라이브에 저장된 문제 파일에 .soln 파일이 생성 되었다면 코드 실행이 정상적으로 이루어진 것!**